# Video Understanding

In this example, we will generate a description of a video using `Qwen2-VL`, `Qwen2-5-VL`, `LLava`, and `Idefics3`, with more models coming soon.

This feature is currently in beta, may not work as expected.



## Install Dependencies

In [ ]:
!pip install -U mlx-vlm

## Import Dependencies

In [1]:
from pprint import pprint
from mlx_vlm import load
from mlx_vlm.utils import generate
from mlx_vlm.video_generate import process_vision_info

import mlx.core as mx

/opt/homebrew/Caskroom/miniconda/base/envs/mlx_code/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
This is a beta version of the video understanding. It may not work as expected.


In [ ]:
# Load the model and processor
model, processor = load("mlx-community/Qwen2.5-VL-7B-Instruct-4bit")

In [3]:
# Messages containing a video and a text query
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "videos/fastmlx_local_ai_hub.mp4",
                "max_pixels": 360 * 360,
                "fps": 1.0,
            },
            {"type": "text", "text": "Describe this video."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)


numpy reader: video_path=videos/fastmlx_local_ai_hub.mp4, total_frames=1134, video_fps=59.941855343141576, time=0.000s


In [4]:
# Convert inputs to mlx arrays
input_ids = mx.array(inputs['input_ids'])
pixel_values = mx.array(inputs['pixel_values_videos'])
mask = mx.array(inputs['attention_mask'])
image_grid_thw = mx.array(inputs['video_grid_thw'])

kwargs = {
    "image_grid_thw": image_grid_thw,
}

In [9]:
kwargs["video"] = "videos/fastmlx_local_ai_hub.mp4"
kwargs["input_ids"] = input_ids
kwargs["pixel_values"] = pixel_values
kwargs["mask"] = mask
response = generate(model, processor, prompt=text, temperature=0.7, max_tokens=100, **kwargs)

In [10]:
pprint(response)


('The video appears to be a live stream or a recording of a coding session, '
 'likely on a platform like Discord, as indicated by the presence of text '
 "chats and a streamer's interface. The video is primarily focused on a "
 'computer screen displaying a code editor with various programming languages '
 'and snippets of code. The coder seems to be explaining or demonstrating '
 'something related to the code, possibly working through a programming '
 'problem, explaining the logic, or showing the process of solving a problem.\n'
 '\n'
 'Here are some key observations from')


In [ ]:
# open video and play it
from ipywidgets import Video
Video.from_file("videos/fastmlx_local_ai_hub.mp4", width=320, height=240)